## 전처리를 위한 코드 쥬피터 노트북  

## import modules

In [23]:
import os
import pandas as pd
from pytorch_model_summary import summary
import torch

## 목표 : train.csv 파일 새로 만들기  
path, class, dir_name, gender, age, mask_state  

In [5]:
original_csv_path = os.path.join(os.getcwd(), 'dataset', 'train', 'train.csv') # train csv file path
image_dir_path = os.path.join(os.getcwd(), 'dataset', 'train', 'images') # train image dir path

df = pd.read_csv(original_csv_path) # read csv file
result = pd.DataFrame()
img_paths = [] # save for all train image
mask_states = [] # save for all mask state
ages = [] # save for all age
genders = [] # save for all age

for dir_name, age, gender in zip(df['path'], df['age'], df['gender']):
    each_dir_path = os.path.join(image_dir_path,dir_name)
    files_names = os.listdir(each_dir_path)

    for file_name in files_names:
        each_file_path = os.path.join(each_dir_path, file_name)
        img_paths.append(each_file_path)
        ages.append(age)
        genders.append(gender)
        if 'mask' in file_name and 'incorrect' not in file_name:
            mask_states.append('Wear')
        elif 'incorrect' in file_name:
            mask_states.append('Incorrect')
        elif 'normal' in file_name:
            mask_states.append('Not Wear')
        else:
            print(f'{file_name} is image file?')


In [6]:
result['path'] = img_paths
result['mask_state'] = mask_states
result['age'] = ages
result['gender'] = genders

In [7]:
def get_label(mask_state, gender, age):
    gender = gender.lower()
    label = 0
    if mask_state == 'Wear' and gender == 'male' and age<30: # 0:
        label = 0
    elif mask_state == 'Wear' and gender == 'male' and 30<=age<60: # 1
        label = 1
    elif mask_state == 'Wear' and gender == 'male' and age>=60: # 2
        label = 2
    elif mask_state == 'Wear' and gender == 'female' and age<30: # 3
        label = 3
    elif mask_state == 'Wear' and gender == 'female' and 30<=age<60: # 4
        label = 4
    elif mask_state == 'Wear' and gender == 'female' and age>=60: # 5
        label = 5
    elif mask_state == 'Incorrect' and gender == 'male' and age<30: # 6
        label = 6
    elif mask_state == 'Incorrect' and gender == 'male' and 30<=age<60: # 7
        label = 7
    elif mask_state == 'Incorrect' and gender == 'male' and age>=60: # 8
        label = 8
    elif mask_state == 'Incorrect' and gender == 'female' and age<30: # 9
        label = 9
    elif mask_state == 'Incorrect' and gender == 'female' and 30<=age<60: # 10
        label = 10
    elif mask_state == 'Incorrect' and gender == 'female' and age>=60: # 11
        label = 11
    elif mask_state == 'Not Wear' and gender == 'male' and age<30: # 12
        label = 12
    elif mask_state == 'Not Wear' and gender == 'male' and 30<=age<60: # 13
        label = 13
    elif mask_state == 'Not Wear' and gender == 'male' and age>=60: # 14
        label = 14
    elif mask_state == 'Not Wear' and gender == 'female' and age<30: # 15
        label = 15
    elif mask_state == 'Not Wear' and gender == 'female' and 30<=age<60: # 16
        label = 16
    elif mask_state == 'Not Wear' and gender == 'female' and age>=60: # 17
        label = 17
    else:
        raise ValueError
    return label

In [8]:
classes = []
for idx, row in enumerate(result.values.tolist()):
    p, s, a, g = row 
    label = get_label(s, g, a)
    classes.append(label)

In [9]:
result['class'] = classes

In [10]:
result.head(10)

,path,mask_state,age,gender,class
0,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
1,/opt/ml/repo/level1_imageclassification_cv-lev...,Not Wear,45,female,16
2,/opt/ml/repo/level1_imageclassification_cv-lev...,Incorrect,45,female,10
3,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
4,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
5,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
6,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
7,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,52,female,4
8,/opt/ml/repo/level1_imageclassification_cv-lev...,Not Wear,52,female,16
9,/opt/ml/repo/level1_imageclassification_cv-lev...,Incorrect,52,female,10


In [11]:
new_file_root = os.path.join(os.getcwd(), 'dataset', 'train', 'custom_train.csv')
result.to_csv(new_file_root)

In [16]:
result['class'].value_counts()

4     4085
3     3660
0     2745
1     2050
16     817
10     817
15     732
9      732
12     549
6      549
5      545
2      415
7      410
13     410
11     109
17     109
8       83
14      83
Name: class, dtype: int64

## 목표 : Mask, Gender를 training 하기위한 csv 파일 만들기    

In [6]:
os.getcwd()
file_path = os.path.join(os.getcwd(), 'dataset', 'train','custom_train.csv')
target_sheet = pd.read_csv(file_path)

In [28]:
map_dict = {
    'Wear,male':0,
    'Wear,female':1,
    'Incorrect,male':2,
    'Incorrect,female':3,
    'Not Wear,male':4,
    'Not Wear,female':5
}

In [26]:
temp_list = []
for row in zip(target_sheet['mask_state'], target_sheet['gender']):
    # print(row, type(row))
    combined_row = row[0] + ',' +row[1]
    temp_list.append(combined_row)

In [27]:
temp_list

['Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,male',
 'Not Wear,male',
 'Incorrect,male',
 'Wear,male',
 'Wear,male',
 'Wear,male',
 'Wear,male',
 'Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Not Wear,female',
 'Incorrect,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Wear,female',
 'Not Wear,female',
 'Incorrec

In [32]:
mapped_list = list(map(lambda x:map_dict[x],temp_list))
target_sheet['mask_gender_class'] = mapped_list

In [35]:
file_path = os.path.join(os.getcwd(), 'dataset', 'train','mask_gender_sheet.csv')
target_sheet.to_csv(file_path)

In [ ]:
target_sheet

## 목표 : 3head model에 사용가능한 csv 파일 만들기  
```
dict_mask_map = {'Wear' : 0, 'Incorrect' : 1, 'Not Wear' : 2}  
dict_gender_map = {'male' : 0, 'female' : 1}  
```

In [4]:
df = pd.read_csv('./dataset/train/custom_train.csv')
dict_mask_map = {'Wear' : 0, 'Incorrect' : 1, 'Not Wear' : 2}  
dict_gender_map = {'male' : 0, 'female' : 1}  
df

,Unnamed: 0,path,mask_state,age,gender,class
0,0,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
1,1,/opt/ml/repo/level1_imageclassification_cv-lev...,Not Wear,45,female,16
2,2,/opt/ml/repo/level1_imageclassification_cv-lev...,Incorrect,45,female,10
3,3,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
4,4,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,45,female,4
...,...,...,...,...,...,...
18895,18895,/opt/ml/repo/level1_imageclassification_cv-lev...,Incorrect,19,male,6
18896,18896,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,19,male,0
18897,18897,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,19,male,0
18898,18898,/opt/ml/repo/level1_imageclassification_cv-lev...,Wear,19,male,0


In [6]:
new_df = pd.DataFrame()
new_df['path'] = df['path']
new_df['mask_class'] = df['mask_state'].map(dict_mask_map)
new_df['gender_class'] = df['gender'].map(dict_gender_map)
new_df['age'] = df['age']
new_df['origin_label'] = df['class']

In [8]:
new_df.to_csv('./dataset/train/3head_train.csv')

## 목표 : 모델서치

In [2]:
import timm

In [25]:
print(timm.list_models('*convnext*', pretrained=True))

['convnext_atto', 'convnext_atto_ols', 'convnext_base', 'convnext_base_384_in22ft1k', 'convnext_base_in22ft1k', 'convnext_base_in22k', 'convnext_femto', 'convnext_femto_ols', 'convnext_large', 'convnext_large_384_in22ft1k', 'convnext_large_in22ft1k', 'convnext_large_in22k', 'convnext_nano', 'convnext_nano_ols', 'convnext_pico', 'convnext_pico_ols', 'convnext_small', 'convnext_small_384_in22ft1k', 'convnext_small_in22ft1k', 'convnext_small_in22k', 'convnext_tiny', 'convnext_tiny_384_in22ft1k', 'convnext_tiny_hnf', 'convnext_tiny_in22ft1k', 'convnext_tiny_in22k', 'convnext_xlarge_384_in22ft1k', 'convnext_xlarge_in22ft1k', 'convnext_xlarge_in22k']


In [34]:
model = timm.create_model('convnext_tiny',pretrained=True, num_classes=2)
print(summary(model, torch.Tensor(1,3,224,224)))

-------------------------------------------------------------------------------
             Layer (type)         Output Shape         Param #     Tr. Param #
                 Conv2d-1      [1, 96, 56, 56]           4,704           4,704
            LayerNorm2d-2      [1, 96, 56, 56]             192             192
          ConvNeXtStage-3      [1, 96, 56, 56]         237,888         237,888
          ConvNeXtStage-4     [1, 192, 28, 28]         992,256         992,256
          ConvNeXtStage-5     [1, 384, 14, 14]      11,112,960      11,112,960
          ConvNeXtStage-6       [1, 768, 7, 7]      15,470,592      15,470,592
               Identity-7       [1, 768, 7, 7]               0               0
   SelectAdaptivePool2d-8       [1, 768, 1, 1]               0               0
            LayerNorm2d-9       [1, 768, 1, 1]           1,536           1,536
               Flatten-10             [1, 768]               0               0
               Dropout-11             [1, 768]     

-------------------------------------------------------------------------------
             Layer (type)         Output Shape         Param #     Tr. Param #
                 Conv2d-1      [1, 96, 56, 56]           4,704           4,704
            LayerNorm2d-2      [1, 96, 56, 56]             192             192
          ConvNeXtStage-3      [1, 96, 56, 56]         237,888         237,888
          ConvNeXtStage-4     [1, 192, 28, 28]         992,256         992,256
          ConvNeXtStage-5     [1, 384, 14, 14]      11,112,960      11,112,960
          ConvNeXtStage-6       [1, 768, 7, 7]      15,470,592      15,470,592
               Identity-7       [1, 768, 7, 7]               0               0
   SelectAdaptivePool2d-8       [1, 768, 1, 1]               0               0
            LayerNorm2d-9       [1, 768, 1, 1]           1,536           1,536
               Flatten-10             [1, 768]               0               0
               Dropout-11             [1, 768]     

In [36]:
print(timm.list_models('*dense*', pretrained=True))

['densenet121', 'densenet161', 'densenet169', 'densenet201', 'densenetblur121d', 'tv_densenet121']


In [38]:
model = timm.create_model('densenet161', pretrained=True, num_classes=2)
print(summary(model, torch.Tensor(1,3,224,224)))

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /opt/ml/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


---------------------------------------------------------------------------------
              Layer (type)          Output Shape         Param #     Tr. Param #
                  Conv2d-1     [1, 96, 112, 112]          14,112          14,112
          BatchNormAct2d-2     [1, 96, 112, 112]             192             192
               MaxPool2d-3       [1, 96, 56, 56]               0               0
              DenseBlock-4      [1, 384, 56, 56]         751,392         751,392
          BatchNormAct2d-5      [1, 384, 56, 56]             768             768
                  Conv2d-6      [1, 192, 56, 56]          73,728          73,728
               AvgPool2d-7      [1, 192, 28, 28]               0               0
              DenseBlock-8      [1, 768, 28, 28]       2,061,504       2,061,504
          BatchNormAct2d-9      [1, 768, 28, 28]           1,536           1,536
                 Conv2d-10      [1, 384, 28, 28]         294,912         294,912
              AvgPool2d-11 